In [1]:
import warnings
from collections import namedtuple
from pathlib import Path

import keras.backend as K
import numpy as np
import pandas as pd
from keras.layers import InputLayer, Dense, ReLU
from keras.models import Sequential
from keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

In [2]:
base_path = Path('dataset/dataset_versions')

datasets = {}
datasets_names = (
    'bfill_ffill',
    'linear_interpolation',
    'cubic_interpolation',
    'quadratic_interpolation',
    'polynomial_5_interpolation',
    'polynomial_7_interpolation',
    'polynomial_9_interpolation',
    'polynomial_11_interpolation',
)
for dataset_name in datasets_names:
    dataset = pd.read_excel(base_path / f'{dataset_name}_rescaled_dataset.xlsx')
    datasets[dataset_name] = dataset.iloc[:, 1:]

In [3]:
test_size = 0.2
seed = 7
target_feature_name = 'GDP per capita (current US$)'

SplittedDataset = namedtuple('SplittedDataset', ['name', 'x_train', 'y_train', 'x_test', 'y_test'])
splited_datasets = []

for dataset_name, dataset in datasets.items():
    model = dict()
    model['name'] = dataset_name
    data_x = dataset.drop([target_feature_name], axis=1)
    data_y = dataset[target_feature_name]
    model['x_train'], model['x_test'], model['y_train'], model['y_test'] = train_test_split(data_x, data_y, test_size=test_size, random_state=seed)
    splited_datasets.append(SplittedDataset(model['name'], model['x_train'],  model['y_train'], model['x_test'], model['y_test']))

In [4]:
def r2_score_custom(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    r2 = 1 - SS_res/(SS_tot + K.epsilon())
    return r2

def r2_loss(y_true, y_pred):
    return -r2_score_custom(y_true, y_pred)

First model

In [5]:
def create_model():
    model = Sequential()
    input_shape = (dataset.x_train.shape[1], )
    model.add(InputLayer(input_shape=input_shape))
    model.add(Dense(64))
    model.add(ReLU())
    model.add(Dense(1024))
    model.add(ReLU())
    model.add(Dense(1))
    optimizer_model = Adam()
    model.compile(optimizer=optimizer_model, loss=r2_loss)
    return model

for dataset in splited_datasets:
    regressor = KerasRegressor(build_fn=create_model, epochs=50, batch_size=32)
    regressor.fit(dataset.x_train, dataset.y_train)
    test_predict = regressor.predict(dataset.x_test)
    rmse = np.sqrt(mean_squared_error(dataset.y_test, test_predict))
    r2 = r2_score(dataset.y_test, test_predict)
    print(r2, rmse)

Epoch 1/50
7/7 [==============================] - 1s 3ms/step - loss: 3.2993
Epoch 2/50
7/7 [==============================] - 0s 4ms/step - loss: -0.1050
Epoch 3/50
7/7 [==============================] - 0s 3ms/step - loss: -0.5070
Epoch 4/50
7/7 [==============================] - 0s 2ms/step - loss: -0.7486
Epoch 5/50
7/7 [==============================] - 0s 3ms/step - loss: -0.8429
Epoch 6/50
7/7 [==============================] - 0s 2ms/step - loss: -0.8311
Epoch 7/50
7/7 [==============================] - 0s 2ms/step - loss: -0.8866
Epoch 8/50
7/7 [==============================] - 0s 2ms/step - loss: -0.8573
Epoch 9/50
7/7 [==============================] - 0s 2ms/step - loss: -0.8865
Epoch 10/50
7/7 [==============================] - 0s 2ms/step - loss: -0.9084
Epoch 11/50
7/7 [==============================] - 0s 2ms/step - loss: -0.9392
Epoch 12/50
7/7 [==============================] - 0s 2ms/step - loss: -0.9149
Epoch 13/50
7/7 [==============================] - 0s 2ms/step